In [1]:
import socket
import os
from _thread import *
from random import randrange, getrandbits
from math import gcd

### Generating big prime numbers using Miller Rabin Test
<a href='https://medium.com/@ntnprdhmm/how-to-generate-big-prime-numbers-miller-rabin-49e6e6af32fb'>Source</a>

In [2]:
def is_prime(n, k=128):
    """ Test if a number is prime
        Args:
            n -- int -- the number to test
            k -- int -- the number of tests to do
        return True if n is prime
    """
    # Test if n is not even.
    # But care, 2 is prime !
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # do k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True

def generate_prime_candidate(length):
    """ Generate an odd integer randomly
        Args:
            length -- int -- the length of the number to generate, in bits
        return a integer
    """
    # generate random bits
    p = getrandbits(length)
    # apply a mask to set MSB and LSB to 1
    p |= (1 << length - 1) | 1
    return p
def generate_prime_number(length=1024):
    """ Generate a prime
        Args:
            length -- int -- length of the prime to generate, in          bits
        return a prime
    """
    p = 4
    # keep generating while the primality test fail
    while not is_prime(p, 128):
        p = generate_prime_candidate(length)
    return p


### Key Generation

In [3]:
def generate_public_key(phi):
    e = phi-1
    while True:
        if gcd(e, phi) == 1:
            return e
        e -= 1

In [4]:
def generate_key():
    p = generate_prime_number()
    q = generate_prime_number()
    n = p * q
    phi = (p-1) * (q-1)
    e = generate_public_key(phi)
    d = pow(e, -1, phi)
    return (e, n), (d, n)

### Encryption

In [5]:
def encrypt(pk, plaintext):
    # Unpack the key into it's components
    key, n = pk
    # Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [pow(ord(char), key, n) for char in plaintext]
    # Return the array of bytes
    return cipher

### Decryption

In [6]:
def decrypt(pk, ciphertext):
    # Unpack the key into its components
    key, n = pk
    # Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr(pow(char, key, n)) for char in ciphertext]
    # Return the array of bytes as a string
    return ''.join(plain)

In [7]:

public_key, private_key = generate_key()
print(public_key, private_key)

(17025265256862057249099363879306690559718147044674702676092955282953338268607574829710947544104498676712270620679183437639931490271475064439448943259844184731717802817403713512756720393643146668314842687911979311170568626148814589366824400339324388202198387349394709953751485397414845609709351168917762909172148925977693218622295260970875073267600691880559467021443952741955710983385807693442602620800784789580943648874940700453009640991321840511437881170451757436800905833598727204360920908491852637517014539697498005674448255664992948076690069304624272776555341536655237907980441010174610797254206448346462950514919, 17025265256862057249099363879306690559718147044674702676092955282953338268607574829710947544104498676712270620679183437639931490271475064439448943259844184731717802817403713512756720393643146668314842687911979311170568626148814589366824400339324388202198387349394709953751485397414845609709351168917762909172410184249216870049128188503478862093127156208631633941925010073597577634

In [ ]:
# print('Ciphertext: ', encrypt(pk, 'The quick brown fox jumps over the lazy dog.'))
# print('Plaintext: ', decrypt(sk, encrypt(pk, 'The quick brown fox jumps over the lazy dog.')))

In [1]:
import socket
ClientMultiSocket = socket.socket()
host = '127.0.0.1'
port = 2004
print('Waiting for connection response')
try:
    ClientMultiSocket.connect((host, port))
except socket.error as e:
    print(str(e))
res = ClientMultiSocket.recv(1024)
while True:
    Input = input('Hey there: ')
    ClientMultiSocket.send(str.encode(Input))
    res = ClientMultiSocket.recv(1024)
    print(res.decode('utf-8'))
ClientMultiSocket.close()

Waiting for connection response
Hey there: yo wassup
Server message: yo wassup
Hey there: hello world
Server message: hello world


KeyboardInterrupt: Interrupted by user